In [30]:
# 使用 PyMySQL 來將下載讀取完成的資料寫入 MySQL 資料庫保存，並且再讀取出來



# 1. 先將資料下載並做成 csv 格式資料

import requests
import io
import csv

response = requests.get("https://data.kcg.gov.tw/File/DirectDownload/d8e11066-81e7-44ee-963d-4fdc37055cfc")

io_file = io.StringIO(response.content.decode())

csv_file = list(csv.reader(io_file))

data_column , *data_row = csv_file

# print(data_column)

# for row in data_row : 

#     print(row)



# 2. 再使用 PyMySQL 對資料庫做存取和異動 ( Insert , Update , Delete )

import pymysql

# 記得就是先建立連線 connection，再建立指標 cursor

connection = pymysql.connect(

    host="127.0.0.1",
    user="root",
    password="12345",
    database="example",
    charset="utf8",
    autocommit=False

)

cursor = connection.cursor()

# 就是打以前 SQL 語言用的一樣增刪修指令

sql = "INSERT INTO new_table (Seq,年度,家數,實際住用客房數,可供住用客房數,住用率) VALUES (%s,%s,%s,%s,%s,%s)"

# 再做 execute 移動指標並確認異動 commit

try : 

    result = cursor.executemany(sql,data_row)

    connection.commit()

except Exception as e : 

    connection.rollback()

    print("變動失敗",e)

    cursor.close()

    connection.close()

# 記得要把連線關掉，不然資料庫會不能做任何執行動作

cursor.close()

connection.close()



# 3. 再使用 PyMySQL 來讀取資料庫和資料 ( Select )

connection = pymysql.connect(

    host="localhost",
    user="root",
    password="12345",
    database="example",
    charset="utf8"

)

cursor = connection.cursor()

# 打 SQL 語言的查詢命令 ( 用之前得到的筆數來查詢最後幾筆是不是那些資料 )

sql = f"SELECT * FROM new_table ORDER BY Seq DESC LIMIT {result}"

# 再做 execute 和抓資料 fetch 

cursor.execute(sql)

results = cursor.fetchall()

for result in results : 

    print(result)

cursor.close()

connection.close()





('8', '2024', '10', '708812', '1276212', '55.54')
('7', '2023', '10', '706827', '1284195', '55.04')
('6', '2022', '11', '652625', '1441385', '45.28')
('5', '2021', '11', '468923', '1442085', '32.52')
('4', '2020', '10', '493418', '1192606', '41.37')
('3', '2019', '11', '914766', '1387994', '65.91')
('2', '2018', '12', '853837', '1502714', '56.82')
('1', '2017', '12', '897389', '1443836', '62.15')
